# Libraries

In [281]:
import requests

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None # removing some warnings
pd.set_option('display.max_columns', None) # display all columns in DF
import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns

from ast import literal_eval

import time
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdate

from linearmodels import PooledOLS
import statsmodels.api as sm



# Loading Data

In [282]:
df_reg_org = pd.read_csv('raw_consolidation_data.csv', sep = '~')

In [283]:
df_reg_org.head(2)

,date,unique_id,marketing_category,brand_name,generic_name,labeler_name,pharm_class,is_original_packager,units_reimbursed,total_amount_reimbursed_adj,price_per_unit_adj
0,1991-01-01,acetaminophen-tablet-oral-300 mg/1,ANDA,Acetaminophen and Codeine Phosphate,ACETAMINOPHEN,"Teva Pharmaceuticals USA, Inc.","['Full Opioid Agonists [MoA]', 'Opioid Agonist...",True,2.599944e+06,1.554268e+06,0.604724
1,1991-01-01,acetaminophen-tablet-oral-325 mg/1,ANDA,"butalbital, acetaminophen and caffeine",ACETAMINOPHEN,"Mikart, LLC","['Barbiturate [EPC]', 'Barbiturates [CS]', 'Ce...",True,2.762276e+05,1.785151e+05,0.646261


# Data manipulation

Removing labeller that are original packager

In [284]:
df_reg = df_reg_org.loc[df_reg_org['is_original_packager'] == True]

In [285]:
print(len(df_reg))

551764


### Assigning order of entrance for each unique drug

In [286]:
df_temp = df_reg.sort_values(by = ['unique_id', 'date', 'marketing_category'], ascending = [False, True, False], ignore_index = True) # Sorting

df_temp = df_temp.drop_duplicates(subset = ['unique_id', 'labeler_name'], keep = 'first') # Finding the first entrance for each labeller 

df_temp['labeler_name_count'] = df_temp.groupby((df_temp['unique_id'] != df_temp['unique_id'].shift(1)).cumsum()).cumcount() + 1 # Running count for each drung

df_reg = pd.merge(df_temp[['unique_id', 'labeler_name', 'labeler_name_count']], df_reg, on = ['unique_id', 'labeler_name'], how = 'right') # Merging with org. data

### Assigning a dummy for first entrance and NDA

In [287]:
df_temp = df_reg.loc[(df_reg['labeler_name_count'] == 1) & (df_reg['marketing_category'] == 'NDA')] # Filtering

df_temp = df_temp.sort_values(by = ['date', 'unique_id'], ascending = [True, False], ignore_index = True) # Sorting
df_temp = df_temp.drop_duplicates(subset = ['unique_id'], keep = 'first') # Removing dublicates

df_temp['first_nda'] = 1 # Adding dummy

df_reg = pd.merge(df_temp[['unique_id', 'first_nda']], df_reg, on = ['unique_id'], how = 'right') # Merging with reg. data

df_reg['first_nda'] = df_reg['first_nda'].fillna(0)

Removing unique drug where the first entrance is not a NDA

In [288]:
df_reg = df_reg.loc[(df_reg['first_nda'] == 1)]

### Assigning a running count for each unique drug from the start

In [289]:
df_temp = df_reg[df_reg['labeler_name_count'] == 1]

df_temp = df_temp.drop_duplicates(subset = ['date', 'unique_id'], keep = 'first') # Removing dublicates

df_temp['running_count_from_start'] = df_temp.groupby((df_temp['unique_id'] != df_temp['unique_id'].shift(1)).cumsum()).cumcount() + 1 # Running count for each drung

df_reg = pd.merge(df_temp[['date', 'unique_id', 'running_count_from_start']], df_reg, on = ['date', 'unique_id',], how = 'right')

### Assigning a running count for each unique drug from second entrance

In [290]:
df_temp = df_reg[df_reg['labeler_name_count'] >= 2]

df_temp = df_temp.drop_duplicates(subset = ['date', 'unique_id'], keep = 'first') # Removing dublicates

df_temp['running_count_from_second_entrance'] = df_temp.groupby((df_temp['unique_id'] != df_temp['unique_id'].shift(1)).cumsum()).cumcount() + 1 # Running count for each drung

df_reg = pd.merge(df_temp[['date', 'unique_id', 'running_count_from_second_entrance']], df_reg, on = ['date', 'unique_id',], how = 'right')

df_reg['running_count_from_second_entrance'] = df_reg['running_count_from_second_entrance'].fillna(0)

### Assigning a running count for each unique drug from third entrance

In [291]:
df_temp = df_reg[df_reg['labeler_name_count'] >= 3]

df_temp = df_temp.drop_duplicates(subset = ['date', 'unique_id'], keep = 'first') # Removing dublicates

df_temp['running_count_from_third_entrance'] = df_temp.groupby((df_temp['unique_id'] != df_temp['unique_id'].shift(1)).cumsum()).cumcount() + 1 # Running count for each drung

df_reg = pd.merge(df_temp[['date', 'unique_id', 'running_count_from_third_entrance']], df_reg, on = ['date', 'unique_id',], how = 'right')

df_reg['running_count_from_third_entrance'] = df_reg['running_count_from_third_entrance'].fillna(0)

### Assigning a running count for each unique drug from fourth entrance

In [292]:
df_temp = df_reg[df_reg['labeler_name_count'] >= 4]

df_temp = df_temp.drop_duplicates(subset = ['date', 'unique_id'], keep = 'first') # Removing dublicates

df_temp['running_count_from_fourth_entrance'] = df_temp.groupby((df_temp['unique_id'] != df_temp['unique_id'].shift(1)).cumsum()).cumcount() + 1 # Running count for each drung

df_reg = pd.merge(df_temp[['date', 'unique_id', 'running_count_from_fourth_entrance']], df_reg, on = ['date', 'unique_id',], how = 'right')

df_reg['running_count_from_fourth_entrance'] = df_reg['running_count_from_fourth_entrance'].fillna(0)

### Assigning a total count of the labelers at the start (to exclude unique drug with multiple labelers from the start)

In [293]:
df_temp = df_reg.loc[(df_reg['running_count_from_start'] == 1)]
          
df_temp = df_temp.groupby(['unique_id'])['first_nda'].sum()
df_temp = df_temp.reset_index()
df_temp = df_temp.rename(columns = {"first_nda": "number_of_first_entrance"})

df_temp = df_temp.loc[(df_temp['number_of_first_entrance'] > 1)]

df_reg = pd.merge(df_temp[['unique_id', 'number_of_first_entrance']], df_reg, on = ['unique_id'], how = 'right') # Merging with reg. data

df_reg['number_of_first_entrance'] = df_reg['number_of_first_entrance'].fillna(1)

Removing unique drug which has multiple entrances at start

In [294]:
df_reg = df_reg.loc[(df_reg['number_of_first_entrance'] == 1)]

### Assigning a running count of generic labelers

In [295]:
df_temp = df_reg[['date', 'unique_id', 'labeler_name', 'labeler_name_count']]

df_temp = df_temp.groupby(['date', 'unique_id'])

df_temp = df_temp.max()

df_temp = df_temp.reset_index()

df_temp['running_count_generics'] = df_temp['labeler_name_count'] - 1 # Minus by one because org. labeler do not count

df_reg = pd.merge(df_temp[['date', 'unique_id', 'running_count_generics']], df_reg, on = ['date', 'unique_id'], how = 'right') # Merging with org. data

### Assigning a running count of substitutes labelers

Unpacking pharm class to obtain EPC and MoA

In [296]:
df_temp = pd.DataFrame(df_reg[['unique_id', 'pharm_class']])

df_temp = df_temp.drop_duplicates(subset = ['unique_id', 'pharm_class'])

df_temp['pharm_class'] = df_temp['pharm_class'].apply(literal_eval)
df_temp = df_temp.explode('pharm_class')

df_temp['pharm_class_type'] = df_temp['pharm_class'].apply(lambda x: str(x)[-5:])
df_temp['pharm_class_type'] = df_temp['pharm_class_type'].str.replace(r'[][]', '', regex=True)

df_temp_EPC = df_temp[df_temp['pharm_class_type'] == 'EPC']
df_temp_MoA = df_temp[df_temp['pharm_class_type'] == 'MoA']

df_temp_EPC = df_temp_EPC.rename(columns = {"pharm_class": "pharm_class_EPC"})
df_temp_MoA = df_temp_MoA.rename(columns = {"pharm_class": "pharm_class_MoA"})

df_temp_EPC = df_temp_EPC.drop_duplicates(subset = ['unique_id', 'pharm_class_EPC'])
df_temp_MoA = df_temp_MoA.drop_duplicates(subset = ['unique_id', 'pharm_class_MoA'])

df_pharm_class_type = pd.merge(df_temp_EPC, df_temp_MoA,  on = 'unique_id', how = 'left')

Adding a route

In [297]:
df_openFDA_NDC = pd.read_csv('raw_openFDA_NDC_data.csv', sep = '~')

In [298]:
df_temp = df_openFDA_NDC.drop_duplicates(subset = ['unique_id'])
df_pharm_class_type = pd.merge(df_pharm_class_type, df_temp[['unique_id', 'route']],  on = 'unique_id', how = 'left')

Creating a unique id for substitutes

In [299]:
df_pharm_class_type['unique_substitute'] = df_pharm_class_type['pharm_class_EPC'].astype(str) + '-' + df_pharm_class_type['route'].astype(str) + '-' + df_pharm_class_type['pharm_class_MoA'].astype(str)
df_pharm_class_type = df_pharm_class_type[['unique_id', 'unique_substitute']]

df_pharm_class_type = df_pharm_class_type.drop_duplicates(subset = ['unique_id'])

Merging with org. data

In [300]:
df_reg = pd.merge(df_reg, df_pharm_class_type[['unique_id', 'unique_substitute']],  on = 'unique_id', how = 'outer')

Calculating number of substitutes

In [301]:
df_temp = df_reg.groupby(['date', 'unique_substitute'])['unique_id'].count()
df_temp = df_temp.reset_index()

df_temp['running_count_unique_substitute'] = df_temp['unique_id'] - 1

df_reg = pd.merge(df_reg, df_temp[['date', 'unique_substitute', 'running_count_unique_substitute']],  on = ['date', 'unique_substitute'], how = 'left')

### Assigning a running count where the first entrance is the starting point

Finding the running count from the start for unique drugs

In [302]:
df_temp = df_reg[['date', 'unique_id', 'running_count_from_start', 'running_count_from_second_entrance']]

df_temp['col_temp'] = df_temp['running_count_from_second_entrance'].map(lambda x: True if (x == 1.0)  else False)

df_temp = df_temp[df_temp['col_temp'] == True]
df_temp = df_temp.drop(columns=['col_temp'])
df_temp = df_temp.drop_duplicates(subset=['unique_id'])
df_temp = df_temp.rename(columns = {"running_count_from_start": "col_temp"})
df_temp = df_temp[['unique_id', 'col_temp']]

Merging with org. data

In [303]:
df_reg = pd.merge(df_temp[['unique_id', 'col_temp']], df_reg, on = ['unique_id',], how = 'right')

Calculationg the running count

In [304]:
df_reg['col_temp'] = df_reg['col_temp'].fillna(0)
df_reg['col_temp'] = df_reg['running_count_from_start'] - df_reg['col_temp']

df_reg['running_count_event'] = df_reg['col_temp'].where(df_reg['col_temp'] != df_reg['running_count_from_start'])

df_temp = df_temp.drop(columns=['col_temp'])

### Calculating the quarterly total amount reimbursed and units reimbursed for all labelers

In [305]:
df_temp = df_reg.set_index(['date', 'unique_id']).groupby(level = ['date', 'unique_id'])[['units_reimbursed', 'total_amount_reimbursed_adj']].agg('sum')
df_temp = df_temp.reset_index()

df_temp = df_temp.rename(columns = {"units_reimbursed": "units_reimbursed_sum", "total_amount_reimbursed_adj": "total_amount_reimbursed_adj_sum"})

df_reg = pd.merge(df_temp[['date', 'unique_id', 'units_reimbursed_sum', 'total_amount_reimbursed_adj_sum']], df_reg, on = ['date', 'unique_id',], how = 'right')

### Calculating the generic labelers' share of quarterly total amount reimbursed and units reimbursed

In [306]:
df_temp = df_reg[df_reg['labeler_name_count'] >= 2]

df_temp = df_temp.set_index(['date', 'unique_id']).groupby(level = ['date', 'unique_id'])[['units_reimbursed', 'total_amount_reimbursed_adj']].agg('sum')
df_temp = df_temp.reset_index()

df_temp = df_temp.rename(columns = {"units_reimbursed": "generic_units_reimbursed_sum", "total_amount_reimbursed_adj": "generic_total_amount_reimbursed_adj_sum"})

df_reg = pd.merge(df_temp[['date', 'unique_id', 'generic_units_reimbursed_sum', 'generic_total_amount_reimbursed_adj_sum']], df_reg, on = ['date', 'unique_id',], how = 'right')

df_reg['generic_units_reimbursed_sum'] = df_reg['generic_units_reimbursed_sum'].fillna(0)
df_reg['generic_total_amount_reimbursed_adj_sum'] = df_reg['generic_total_amount_reimbursed_adj_sum'].fillna(0)

In [307]:
df_reg['generic_share_units_reimbursed_sum'] = df_reg['generic_units_reimbursed_sum'] / df_reg['units_reimbursed_sum']
df_reg['generic_share_total_amount_reimbursed_adj_sum'] = df_reg['generic_total_amount_reimbursed_adj_sum'] / df_reg['total_amount_reimbursed_adj_sum']

df_reg['generic_share_units_reimbursed_sum'] = df_reg['generic_share_units_reimbursed_sum'].fillna(0)
df_reg['generic_share_total_amount_reimbursed_adj_sum'] = df_reg['generic_share_total_amount_reimbursed_adj_sum'].fillna(0)

# Check for issue

In [308]:
print(len(df_reg))
df_reg_issue = df_reg.drop_duplicates(subset = ['unique_id', 'date', 'labeler_name'])
print(len(df_reg))

225135
225135


# Downloading Data

In [310]:
df_reg.to_csv('output_regression_org.csv', sep = '~', index = False)

# Loading Data

In [311]:
df_reg = pd.read_csv('output_regression_org.csv', sep = '~')

# Regression for generic paradox

## For org. labeler

In [312]:
df_temp = df_reg
len(df_temp)

225135

Filtering on orig. labeler

In [313]:
df_temp = df_temp[df_temp['marketing_category'] == "NDA"]
len(df_temp)

139197

Calculating the log of price per unit

In [314]:
df_temp['ln_price_per_unit_adj'] = np.log(df_temp['price_per_unit_adj'])

Calculating the share of units reimbursed for the org. labeler

In [315]:
df_temp['org_share_units_reimbursed_sum'] = 1 - df_temp['generic_share_units_reimbursed_sum']

Change the format of the date to a integer

In [316]:
df_temp['date_int'] = df_temp['date'].str.replace('-', '')
df_temp['date_int'] = df_temp['date_int'].apply(pd.to_numeric)

Setting the index

In [317]:
df_OLS_ndc_gp = df_temp.set_index(['unique_id', 'date_int'])

Estimating the regression

In [318]:
endog = df_OLS_ndc_gp['ln_price_per_unit_adj']
exog_vars = ['running_count_generics', 'running_count_unique_substitute', 'org_share_units_reimbursed_sum', 'running_count_from_second_entrance']
exog = sm.add_constant(df_OLS_ndc_gp[exog_vars])

mod = PooledOLS(endog, exog)
pooled_res = mod.fit()
print(pooled_res)

C:\Users\sebas\anaconda3\lib\site-packages\linearmodels\shared\exceptions.py:37: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                            PooledOLS Estimation Summary                           
Dep. Variable:     ln_price_per_unit_adj   R-squared:                        0.0314
Estimator:                     PooledOLS   R-squared (Between):             -0.0019
No. Observations:                 136950   R-squared (Within):              -0.1151
Date:                   Wed, Mar 16 2022   R-squared (Overall):              0.0314
Time:                           15:32:05   Log-likelihood                -3.142e+05
Cov. Estimator:               Unadjusted                                           
                                           F-statistic:                      1108.4
Entities:                           2951   P-value                           0.0000
Avg Obs:                          46.408   Distribution:                F(4,136945)
Min Obs:                          1.0000                                           
Max Obs:                          421.00   F-statistic (robust):            

Counting number of unique drugs

In [321]:
df_temp.head()

,date,unique_id,generic_units_reimbursed_sum,generic_total_amount_reimbursed_adj_sum,units_reimbursed_sum,total_amount_reimbursed_adj_sum,col_temp,running_count_generics,number_of_first_entrance,running_count_from_fourth_entrance,running_count_from_third_entrance,running_count_from_second_entrance,running_count_from_start,first_nda,labeler_name,labeler_name_count,marketing_category,brand_name,generic_name,pharm_class,is_original_packager,units_reimbursed,total_amount_reimbursed_adj,price_per_unit_adj,unique_substitute,running_count_unique_substitute,running_count_event,generic_share_units_reimbursed_sum,generic_share_total_amount_reimbursed_adj_sum
0,1991-01-01,acetohydroxamic acid-tablet-oral-250 mg/1,0.0,0.0,18592.0,50898.137194,1.0,0,1.0,0.0,0.0,0.0,1.0,1.0,Mission Pharmacal Company,1,NDA,Lithostat,ACETOHYDROXAMIC ACID,"['Urease Inhibitor [EPC]', 'Urease Inhibitors ...",True,18592.0,50898.137194,2.737636,Urease Inhibitor [EPC]-ORAL-Urease Inhibitors ...,0.0,NaN,0.0,0.0
1,1991-04-01,acetohydroxamic acid-tablet-oral-250 mg/1,0.0,0.0,18839.0,51893.063430,2.0,0,1.0,0.0,0.0,0.0,2.0,1.0,Mission Pharmacal Company,1,NDA,Lithostat,ACETOHYDROXAMIC ACID,"['Urease Inhibitor [EPC]', 'Urease Inhibitors ...",True,18839.0,51893.063430,2.754555,Urease Inhibitor [EPC]-ORAL-Urease Inhibitors ...,0.0,NaN,0.0,0.0
2,1991-07-01,acetohydroxamic acid-tablet-oral-250 mg/1,0.0,0.0,28418.0,76648.158195,3.0,0,1.0,0.0,0.0,0.0,3.0,1.0,Mission Pharmacal Company,1,NDA,Lithostat,ACETOHYDROXAMIC ACID,"['Urease Inhibitor [EPC]', 'Urease Inhibitors ...",True,28418.0,76648.158195,2.697169,Urease Inhibitor [EPC]-ORAL-Urease Inhibitors ...,0.0,NaN,0.0,0.0
3,1991-10-01,acetohydroxamic acid-tablet-oral-250 mg/1,0.0,0.0,34263.0,88082.844442,4.0,0,1.0,0.0,0.0,0.0,4.0,1.0,Mission Pharmacal Company,1,NDA,Lithostat,ACETOHYDROXAMIC ACID,"['Urease Inhibitor [EPC]', 'Urease Inhibitors ...",True,34263.0,88082.844442,2.570786,Urease Inhibitor [EPC]-ORAL-Urease Inhibitors ...,0.0,NaN,0.0,0.0
4,1992-01-01,acetohydroxamic acid-tablet-oral-250 mg/1,0.0,0.0,32343.0,80168.781465,5.0,0,1.0,0.0,0.0,0.0,5.0,1.0,Mission Pharmacal Company,1,NDA,Lithostat,ACETOHYDROXAMIC ACID,"['Urease Inhibitor [EPC]', 'Urease Inhibitors ...",True,32343.0,80168.781465,2.478706,Urease Inhibitor [EPC]-ORAL-Urease Inhibitors ...,0.0,NaN,0.0,0.0


In [322]:
df_temp_count = df_temp.groupby(['date'])['unique_id'].count()
df_temp_count = df_temp_count.reset_index()
print('Tabel: ', df_temp_count.head())
print('Quarterly Average: ', df_temp_count['unique_id'].mean())
print('Total: ', df_temp['unique_id'].nunique())
print('Total (generic_name): ', df_temp['generic_name'].nunique())

Tabel:           date  unique_id
0  1991-01-01        145
1  1991-04-01        161
2  1991-07-01        169
3  1991-10-01        190
4  1992-01-01        209
Quarterly Average:  1860.6198347107438
Total:  3012
Total (without strength):  951


# Regression for probability of entrance

### Second entrance

In [323]:
df_temp = df_reg

Assign a dummy for second entrance

In [324]:
df_temp['second_entrance'] = df_temp['labeler_name_count'].apply(lambda x: 1 if x == 2 else 0)
df_temp['second_entrance'] = df_temp['second_entrance'].apply(pd.to_numeric)

Calculating the log of the sum of total amount reimbursed

In [325]:
df_temp['ln_total_amount_reimbursed_adj_sum'] = np.log(df_temp['total_amount_reimbursed_adj_sum'])

Change the format of the date to a integer

In [326]:
df_temp['date'] = df_temp['date'].str.replace('-', '')
df_temp['date'] = df_temp['date'].apply(pd.to_numeric)

Setting index

In [327]:
df_OLS = df_temp.set_index(['unique_id', 'date'])

Estimating the regression

In [328]:
exog_vars = ['ln_total_amount_reimbursed_adj_sum']
exog = sm.add_constant(df_OLS[exog_vars])
endog = df_OLS['second_entrance']

mod = PooledOLS(endog, exog)
pooled_res = mod.fit()
print(pooled_res)

                          PooledOLS Estimation Summary                          
Dep. Variable:        second_entrance   R-squared:                        0.0011
Estimator:                  PooledOLS   R-squared (Between):             -0.2013
No. Observations:              225135   R-squared (Within):              -0.0011
Date:                Wed, Mar 16 2022   R-squared (Overall):              0.0011
Time:                        15:35:50   Log-likelihood                -7.836e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      242.12
Entities:                        3012   P-value                           0.0000
Avg Obs:                       74.746   Distribution:                F(1,225133)
Min Obs:                       1.0000                                           
Max Obs:                       1255.0   F-statistic (robust):             242.12
                            

### Third entrance

In [335]:
df_temp = df_reg

Assign a dummy for second entrance

In [336]:
df_temp['third_entrance'] = df_temp['labeler_name_count'].apply(lambda x: 1 if x == 3 else 0)
df_temp['third_entrance'] = df_temp['third_entrance'].apply(pd.to_numeric)

Calculating the log of the sum of total amount reimbursed

In [337]:
df_temp['ln_total_amount_reimbursed_adj_sum'] = np.log(df_temp['total_amount_reimbursed_adj_sum'])

Setting index

In [340]:
df_OLS = df_temp.set_index(['unique_id', 'date'])

Estimating the regression

In [341]:
exog_vars = ['ln_total_amount_reimbursed_adj_sum']
exog = sm.add_constant(df_OLS[exog_vars])
endog = df_OLS['second_entrance']

mod = PooledOLS(endog, exog)
pooled_res = mod.fit()
print(pooled_res)

                          PooledOLS Estimation Summary                          
Dep. Variable:        second_entrance   R-squared:                        0.0030
Estimator:                  PooledOLS   R-squared (Between):             -0.3199
No. Observations:              225135   R-squared (Within):              -0.0027
Date:                Wed, Mar 16 2022   R-squared (Overall):              0.0030
Time:                        15:38:39   Log-likelihood                -2.992e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      681.76
Entities:                        3012   P-value                           0.0000
Avg Obs:                       74.746   Distribution:                F(1,225133)
Min Obs:                       1.0000                                           
Max Obs:                       1255.0   F-statistic (robust):             681.76
                            